Importing packages

In [1]:
import pyodbc
import pandas as pd
import datetime
from google.cloud import bigquery
import time
import math
from sqlalchemy.engine import create_engine

Importing data from big query using google SDK

## `Description of the problem.`

#### Previously, students accessed the materials for a particular course through a third-party learning platform, which incurred costs for the company every time a student logged in. To mitigate these costs, the company developed an in-house learning paltform called `ODYSSEY` where students can now access their materials directly.

#### An A/B test was conducted to compare the performance of the old system with the new in-house platform. The test group consisted of 279 students from a specific section of an instructor. The goal was to identify a comparable  group of studnets from the remaining sections of the same instructor.

#### Since the test group was predetermined, statistical control was implemented to determine the effectiveness of `ODYSSEY` instead of finding a separate comparable control group.

In [2]:
%%time

# Read in earned credits data from BigQuery tables

bigquery_client = bigquery.Client(project='sei-mdp-datascience-sbx-4d50')

odyssey_statictical_control_cs_query = """

-- actual table names have been ommited and replaced with dummy table names for privacy reasons.

WITH starting_table AS(
    SELECT substr(sc.full_class_id,1,9) AS class_id_section, CONCAT(substr(sc.full_class_id,1,9),' ',sc.class_number) AS class_id_section_number,sc.emplid, sc.term_code, sc.class_id, sc.full_class_id, st.enrolled_aic_count AS aic_clscnt,
            CASE WHEN CONCAT(substr(sc.full_class_id,1,9),' ',sc.class_number) IN ('ENG116005 2029', 'ENG116047 2830', 'ENG116001 2025', 'ENG116046 2829', 'ENG116016 2374', 'ENG116009 2033', 'ENG116004 2028', 'ENG116002 2026') AND primary_instructor_name = 'Gureghian, Sahag' THEN 1 
            WHEN primary_instructor_name = 'Gureghian, Sahag' AND CONCAT(substr(sc.full_class_id,1,9),' ',sc.class_number) NOT IN ('ENG116005 2029', 'ENG116047 2830', 'ENG116001 2025', 'ENG116046 2829', 'ENG116016 2374', 'ENG116009 2033', 'ENG116004 2028', 'ENG116002 2026') THEN 0 END odyssey_flag,
        
            credit_earned_count
    FROM `table 1` sc
    INNER JOIN `table 2` st ON st.emplid = sc.emplid AND st.term_code = sc.term_code 
    INNER JOIN `table 3` dc On dc.full_class_id = sc.full_class_id
    WHERE --sc.class_id = 'ENG116' AND 
    sc.enrolled_aic_count > 0  AND sc.term_code = '1244' AND primary_instructor_name = 'Gureghian, Sahag' AND sc.class_id = 'ENG116'
)  --SELECT * FROM starting_table ORDER BY emplid, term_code, full_class_id
 --SELECT odyssey_flag, COUNT(*) FROM starting_table GROUP BY odyssey_flag

, priority_score_t AS(
  SELECT emplid, term_code, class_id, full_class_id, prediction_timestamp, prediction_week_number, score_for_api
  FROM `table 3`
  WHERE --term_code = '1244' AND class_id = 'ENG116' AND 
  prediction_week_number = 3
  QUALIFY ROW_NUMBER() OVER(PARTITION BY emplid, full_class_id ORDER BY prediction_timestamp DESC) =1
) 

, priority_score_merge AS(
  SELECT st.*, pst.score_for_api
  FROM starting_table st
  LEFT JOIN priority_score_t pst ON pst.emplid = st.emplid AND pst.full_class_id = st.full_class_id
)  --SELECT * FROM priority_score_merge 

, priority_score_aggregated AS(
  SELECT term_code,odyssey_flag, aic_clscnt, credit_earned_count, AVG(score_for_api) AS average_week3_priority_score
  FROM priority_score_merge
  GROUP BY term_code, odyssey_flag,aic_clscnt, credit_earned_count
) SELECT * FROM priority_score_aggregated ORDER BY term_code

"""

query_job = bigquery_client.query(odyssey_statictical_control_cs_query)
odyssey_statictical_control_cs = query_job.to_dataframe()
odyssey_statictical_control_cs.columns = [x.lower() for x in odyssey_statictical_control_cs.columns]
odyssey_statictical_control_cs

C:\Users\Rohan.Azad\AppData\Roaming\Python\Python39\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


CPU times: total: 109 ms
Wall time: 4.49 s


,term_code,odyssey_flag,aic_clscnt,credit_earned_count,average_week3_priority_score
0,1244,0,1,0,0.420164
1,1244,0,3,0,0.479619
2,1244,1,3,1,0.806987
3,1244,1,1,1,0.757513
4,1244,1,2,1,0.765596
5,1244,0,3,1,0.784647
6,1244,0,2,0,0.543877
7,1244,1,1,0,0.599533
8,1244,1,2,0,0.583517
9,1244,0,4,1,0.427771


## `Description of the dataset`.

### `term_code`: the quarter in each the event happend.
### `odyssey_flag`: Whether the student is part of the new in-house learning platform or not. 
### `aic_clscnt`: How many courses they are enrolled in that quarter.
### `average_week3_priority_score`: average predictive model score indicating the probablity of passing the course for that quarter.
### `credit_earned_count`: Target variable. 1 indictaes pass and 0 indicates fail at the end of teh quarter for that course.

## `Data Processing steps`

Check for the data types of columns. The ones that are "object" will need to be converted to dummy variables.

In [3]:
odyssey_statictical_control_cs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   term_code                     12 non-null     object 
 1   odyssey_flag                  12 non-null     Int64  
 2   aic_clscnt                    12 non-null     Int64  
 3   credit_earned_count           12 non-null     Int64  
 4   average_week3_priority_score  12 non-null     float64
dtypes: Int64(3), float64(1), object(1)
memory usage: 644.0+ bytes


term_code is "object" type. Changing the "object" type to "category".

In [4]:
odyssey_statictical_control_cs['term_code'] = odyssey_statictical_control_cs['term_code'].astype('category')
odyssey_statictical_control_cs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   term_code                     12 non-null     category
 1   odyssey_flag                  12 non-null     Int64   
 2   aic_clscnt                    12 non-null     Int64   
 3   credit_earned_count           12 non-null     Int64   
 4   average_week3_priority_score  12 non-null     float64 
dtypes: Int64(3), category(1), float64(1)
memory usage: 676.0 bytes


In [5]:
one_hot_encoded = pd.get_dummies(odyssey_statictical_control_cs[['term_code']])

changing the one_hot_encoded data type to int.

In [6]:
one_hot_encoded = one_hot_encoded.astype(int)

drop the term-code column since that was the category column and join the one_hot_encoded columns to the dataframe.

In [7]:
odyssey_statictical_control_cs_encoded=pd.concat([odyssey_statictical_control_cs.drop(['term_code'], axis =1), one_hot_encoded], axis =1)

## `Model Implementation Steps`

In [8]:
odyssey_statictical_control_cs_encoded

,odyssey_flag,aic_clscnt,credit_earned_count,average_week3_priority_score,term_code_1244
0,0,1,0,0.420164,1
1,0,3,0,0.479619,1
2,1,3,1,0.806987,1
3,1,1,1,0.757513,1
4,1,2,1,0.765596,1
5,0,3,1,0.784647,1
6,0,2,0,0.543877,1
7,1,1,0,0.599533,1
8,1,2,0,0.583517,1
9,0,4,1,0.427771,1


In [9]:
odyssey_statictical_control_cs_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   odyssey_flag                  12 non-null     Int64  
 1   aic_clscnt                    12 non-null     Int64  
 2   credit_earned_count           12 non-null     Int64  
 3   average_week3_priority_score  12 non-null     float64
 4   term_code_1244                12 non-null     int32  
dtypes: Int64(3), float64(1), int32(1)
memory usage: 596.0 bytes


In [10]:
import numpy as np

In [11]:
odyssey_statictical_control_cs_encoded['credit_earned_count'] = odyssey_statictical_control_cs_encoded['credit_earned_count'].astype(np.float64)

In [13]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

In [14]:
X= odyssey_statictical_control_cs_encoded[['odyssey_flag', 'aic_clscnt', 'average_week3_priority_score']]
scaler=StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled=sm.add_constant(X_scaled)
y = odyssey_statictical_control_cs_encoded['credit_earned_count']

In [15]:
model = sm.Logit(y, X_scaled).fit()

         Current function value: 0.000000
         Iterations: 35


c:\Users\Rohan.Azad\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
c:\Users\Rohan.Azad\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [16]:
print(model.summary())

                            Logit Regression Results                           
Dep. Variable:     credit_earned_count   No. Observations:                   12
Model:                           Logit   Df Residuals:                        8
Method:                            MLE   Df Model:                            3
Date:                 Tue, 16 Jul 2024   Pseudo R-squ.:                   1.000
Time:                         14:20:40   Log-Likelihood:            -3.0476e-07
converged:                       False   LL-Null:                       -8.1503
Covariance Type:             nonrobust   LLR p-value:                 0.0009839
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         33.8222   1.48e+06   2.29e-05      1.000    -2.9e+06     2.9e+06
x1           -28.6087   1.75e+06  -1.64e-05      1.000   -3.43e+06    3.43e+06
x2            77.6879   8849.250      0.009 

## `Conclusion`

The model indicates a perfect fit indicated by Pseudo R-squared as 1. 
The coefficients and their standard errors are extremely large which indicates instability in the estimates. 
The p-value being higher than 0.05 for each variable indicates that it is not statistically significant. 

The fact that odyssey_flag has negative coefficient indicates the new-learning platform Odyssey is resulting in lower pass rate. We had done a separate A/B tetsing with test and control group which had also showed that the pass rate for the test group was lower compared to control group.